In [1]:
from tensorflow.keras import models, layers
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/train.csv') #데이터 불러옴
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
df = df.drop(['Name','Ticket'], axis=1) #불필요한 열 삭제

In [5]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,7.2500,NaN,S
1,2,1,1,female,38.0,1,0,71.2833,C85,C
2,3,1,3,female,26.0,0,0,7.9250,NaN,S
3,4,1,1,female,35.0,1,0,53.1000,C123,S
4,5,0,3,male,35.0,0,0,8.0500,NaN,S


In [6]:
df.isnull().sum()/len(df)*100 #빈 값이 얼마나 되는지 확인

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64

In [7]:
# 빈 값 데이터 전처리 (Age와 Embarked는 중앙값)
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

In [8]:
df = df.drop(['Cabin'], axis=1) #cabin은 빈값이 너무 많아 그냥 삭제

In [9]:
df = pd.get_dummies(df, columns=['Sex', 'Embarked'], drop_first=True)

In [10]:
df.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,7.2500,True,False,True
1,2,1,1,38.0,1,0,71.2833,False,False,False
2,3,1,3,26.0,0,0,7.9250,False,False,True
3,4,1,1,35.0,1,0,53.1000,False,False,True
4,5,0,3,35.0,0,0,8.0500,True,False,True


In [11]:
X = df.iloc[:, 2:].values  # 첫 번째 두 열(Survived 포함)을 제외한 나머지 (특성)
y = df.iloc[:, 1].values   # 두 번째 열 (Survived, 타겟)

In [12]:
# 데이터셋 분리: 훈련 데이터 80%, 테스트 데이터 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)

In [14]:
model = models.Sequential()

model.add(layers.Dense(30, input_dim=X_train.shape[1], activation='relu'))
model.add(layers.Dense(12, activation='relu'))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [41]:
history = model.fit(X_train, y_train, epochs=500, batch_size=200, verbose=1, validation_split=0.25)
# history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/500
3/3 [==============================] - 0s 34ms/step - loss: 0.1984 - accuracy: 0.9026 - val_loss: 1.0201 - val_accuracy: 0.8202
Epoch 2/500
3/3 [==============================] - 0s 14ms/step - loss: 0.1961 - accuracy: 0.9120 - val_loss: 1.0133 - val_accuracy: 0.8258
Epoch 3/500
3/3 [==============================] - 0s 14ms/step - loss: 0.1986 - accuracy: 0.8989 - val_loss: 1.0026 - val_accuracy: 0.8034
Epoch 4/500
3/3 [==============================] - 0s 13ms/step - loss: 0.1996 - accuracy: 0.9101 - val_loss: 1.0008 - val_accuracy: 0.8202
Epoch 5/500
3/3 [==============================] - 0s 14ms/step - loss: 0.1961 - accuracy: 0.9064 - val_loss: 1.0052 - val_accuracy: 0.8371
Epoch 6/500
3/3 [==============================] - 0s 14ms/step - loss: 0.1986 - accuracy: 0.9120 - val_loss: 1.0024 - val_accuracy: 0.8034
Epoch 7/500
3/3 [==============================] - 0s 14ms/step - loss: 0.1929 - accuracy: 0.9101 - val_loss: 0.9956 - val_accuracy: 0.8258
Epoch 8/500
3/3 [===

In [42]:
print(X_test.shape)
print(y_test.shape)

(179, 8)
(179,)


In [43]:
score = model.evaluate(X_test, y_test)
print('Test Accuracy: ', score[1])

6/6 [==============================] - 0s 1ms/step - loss: 1.2461 - accuracy: 0.7933
Test Accuracy:  0.7932960987091064
